<a href="https://colab.research.google.com/github/Curro-Gonz/Airport-Signs/blob/main/trainingtfg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrenamiento de YOLOv8 para detectar y clasificar señales de aeropuerto

Instalamos las dependencias necesarias

In [1]:
%%capture
!pip install ultralytics==8.0.196
!pip install roboflow

Obtenemos el directorio en el que vamos a trabajar

In [2]:
import os
HOME = os.getcwd()
print(HOME)

/content


Importamos cv2\_imshow que nos será útil para visualizar los resultados de las operaciones de procesamiento de imágenes.

In [3]:
from google.colab.patches import cv2_imshow

Importamos ultralytics y YOLO para poder entrenar y predecir el modelo

In [4]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

from ultralytics import YOLO

from IPython.display import display, Image

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.1/78.2 GB disk)


Cargamos Drive en el colab para poder tener acceso a las carpetas de Drive para luego guardar los resultados

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Cargamos el link de exportación del *dataset* de roboflow.

Dado que la api\_key es privada, se ha tomado la decisión de no compartirla ya que un mal uso de ella puede conllevar la modificación o pérdida del *dataset* sin que el propietario tenga conocimiento de ello. Esto supone que no se podrá descargar el *dataset*.

In [7]:
%%capture
%cd {HOME}

from roboflow import Roboflow
rf = Roboflow(api_key="AE8R6X7eoyovtmBkFMSl")
project = rf.workspace("curro-p77xe").project("airport-signals")
version = project.version(4)
dataset = version.download("yolov8")


Entrenamos nuestro modelo con 100 épocas

In [ ]:
%%capture
model = YOLO("yolov8n.pt")
results = model.train(data= os.path.join("/content/Airport-signals-4", "data.yaml"), epochs = 100)

Validamos el modelo con la carpeta de validación

In [ ]:
%%capture
!yolo task=detect mode=val model="/content/runs/detect/train/weights/best.pt" data="/content/Airport-signals-4/data.yaml"

Utilizamos nuestro modelo ya entrenado para predecir nuevas imágenes

In [ ]:
%%capture
!yolo task=detect mode=predict model='/content/runs/detect/train/weights/best.pt' conf = 0.5 source='/content/Airport-signals-4/test/images'

Guardamos los resultados en Drive

In [ ]:
import shutil
# Ruta del directorio que quieres copiar
source_dir = '/content/runs'
# Ruta del directorio de destino en Google Drive
dest_dir = '/content/drive/MyDrive/runs'

# Copiar el directorio
shutil.copytree(source_dir, dest_dir)

'/content/drive/MyDrive/runs'

Desplegamos el modelo entrenado en Roboflow

In [ ]:
%%capture
version = project.version(4)
version.deploy("yolov8", "/content/runs/detect/train/weights", "best.pt")